In [286]:
import pandas as pd
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [287]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [288]:
pip install rake-nltk

Note: you may need to restart the kernel to use updated packages.


## Reading dataset

In [289]:
df = pd.read_csv('n_movies.csv')

In [290]:
df

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,"Decades after their 1984 All Valley Karate Tournament bout, a middle-aged Daniel LaRusso and Johnny Lawrence find themselves martial-arts rivals again.","['Ralph Macchio, ', 'William Zabka, ', 'Courtney Henggeler, ', 'Xolo Maridueña']","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of Queen Elizabeth II's reign and the events that shaped the second half of the twentieth century.,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda Staunton, ', 'Matt Smith']","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer Jimmy McGill before his fateful run-in with Walter White and Jesse Pinkman.,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan Banks, ', 'Patrick Fabian']","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,"When a psychiatrist shelters a mysterious cult escapee, her world is turned upside down as the girl's arrival threatens to tear her own family apart.","['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo Celasco, ', 'Madeleine Arthur']","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,"A Street Kid trying to survive in a technology and body modification-obsessed city of the future. Having everything to lose, he chooses to stay alive by becoming an Edgerunner, a Mercenary outlaw also known as a Cyberpunk.","['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi Lo, ', 'Aoi Yûki']","15,413"
...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,"After an experimental gene therapy turns them into monsters, three twenty-somethings band together to hunt down the scientist responsible and force him to make them human again.","['Morgan Taylor Campbell, ', 'Italia Ricci, ', 'Rhianna Jagpal, ', 'Iñaki Godoy']","3,130"
9953,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melissa McBride, ', 'Lauren Cohan']","970,067"
9954,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of Queen Elizabeth II's reign and the events that shaped the second half of the twentieth century.,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda Staunton, ', 'Matt Smith']","199,898"
9955,Supernatural,(2005–2020),TV-14,44 min,"Drama, Fantasy, Horror",8.4,"Two brothers follow their father's footsteps as hunters, fighting evil supernatural beings of many kinds, including monsters, demons and gods that roam the earth.","['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim Beaver, ', 'Misha Collins']","439,601"


In [291]:
df.head()

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,"Decades after their 1984 All Valley Karate Tournament bout, a middle-aged Daniel LaRusso and Johnny Lawrence find themselves martial-arts rivals again.","['Ralph Macchio, ', 'William Zabka, ', 'Courtney Henggeler, ', 'Xolo Maridueña']","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of Queen Elizabeth II's reign and the events that shaped the second half of the twentieth century.,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda Staunton, ', 'Matt Smith']","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer Jimmy McGill before his fateful run-in with Walter White and Jesse Pinkman.,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan Banks, ', 'Patrick Fabian']","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,"When a psychiatrist shelters a mysterious cult escapee, her world is turned upside down as the girl's arrival threatens to tear her own family apart.","['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo Celasco, ', 'Madeleine Arthur']","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,"A Street Kid trying to survive in a technology and body modification-obsessed city of the future. Having everything to lose, he chooses to stay alive by becoming an Edgerunner, a Mercenary outlaw also known as a Cyberpunk.","['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi Lo, ', 'Aoi Yûki']","15,413"


In [292]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9957 entries, 0 to 9956
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        9957 non-null   object 
 1   year         9430 non-null   object 
 2   certificate  6504 non-null   object 
 3   duration     7921 non-null   object 
 4   genre        9884 non-null   object 
 5   rating       8784 non-null   float64
 6   description  9957 non-null   object 
 7   stars        9957 non-null   object 
 8   votes        8784 non-null   object 
dtypes: float64(1), object(8)
memory usage: 700.2+ KB


## Removing Duplicates

In [293]:
df.duplicated(subset ='title').sum()

2045

In [294]:
rows_with_duplicates = df[df.duplicated(subset = 'title')]
rows_with_duplicates

,title,year,certificate,duration,genre,rating,description,stars,votes
287,Heartbreak High,(1994–1999),TV-MA,50 min,"Drama, Romance",7.8,A crash course in the turbulence of being a teen is always on the schedule for the students at Hartley High School in this 1990s series set in Sydney.,"['Callan Mulvey, ', 'Lara Cox, ', 'Emma Roche, ', 'Ada Nicodemou']","3,208"
359,Snowpiercer,(2013),R,126 min,"Action, Sci-Fi, Thriller",7.1,"In a future where a failed climate change experiment has killed all life except for the survivors who boarded the Snowpiercer (a train that travels around the globe), a new class system emerges.","['Bong Joon Ho', '| ', ' Stars:', 'Chris Evans, ', 'Jamie Bell, ', 'Tilda Swinton, ', 'Ed Harris']","359,095"
373,Shameless,(2004–2013),TV-MA,90 min,"Comedy, Crime, Drama",8.0,The lives and relationships of a group of siblings and their estranged father Frank Gallagher on a rough Manchester estate.,"['David Threlfall, ', 'Rebecca Atkinson, ', 'Alice Barry, ', 'Nicky Evans']","16,895"
416,Beauty and the Beast,(1991),G,84 min,"Animation, Family, Fantasy",8.0,A prince cursed to spend his days as a hideous monster sets out to regain his humanity by earning a young woman's love.,"['Gary Trousdale, ', 'Kirk Wise', '| ', ' Stars:', ""Paige O'Hara, "", 'Robby Benson, ', 'Jesse Corti, ', 'Rex Everhart']","452,086"
427,The Stranger,(II) (2022),NaN,117 min,"Crime, Drama, Thriller",7.4,"Two men who meet on a plane and strike up a conversation that turns into friendship. For Henry Teague, worn down by a lifetime of physical labour and crime, this is a dream come true.","['Thomas M. Wright', '| ', ' Stars:', 'Joel Edgerton, ', 'Sean Harris, ', 'Matthew Sunderland, ', 'Ewen Leslie']",85
...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,"After an experimental gene therapy turns them into monsters, three twenty-somethings band together to hunt down the scientist responsible and force him to make them human again.","['Morgan Taylor Campbell, ', 'Italia Ricci, ', 'Rhianna Jagpal, ', 'Iñaki Godoy']","3,130"
9953,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melissa McBride, ', 'Lauren Cohan']","970,067"
9954,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of Queen Elizabeth II's reign and the events that shaped the second half of the twentieth century.,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda Staunton, ', 'Matt Smith']","199,898"
9955,Supernatural,(2005–2020),TV-14,44 min,"Drama, Fantasy, Horror",8.4,"Two brothers follow their father's footsteps as hunters, fighting evil supernatural beings of many kinds, including monsters, demons and gods that roam the earth.","['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim Beaver, ', 'Misha Collins']","439,601"


## Finding columns with duplicate entries

In [295]:
columns_with_duplicates = []
for column in df.columns:
    if df[column].duplicated().any():
        columns_with_duplicates.append(column)

if not columns_with_duplicates:
    print("No columns with duplicate entries.")
else:
    print("Columns with duplicate entries:")
    for column in columns_with_duplicates:
        print(column)

Columns with duplicate entries:
title
year
certificate
duration
genre
rating
description
stars
votes


## Sample check the rows with duplicates

In [296]:
df[df['title'] == 'The Imperfects']

,title,year,certificate,duration,genre,rating,description,stars,votes
8,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,"After an experimental gene therapy turns them into monsters, three twenty-somethings band together to hunt down the scientist responsible and force him to make them human again.","['Morgan Taylor Campbell, ', 'Italia Ricci, ', 'Rhianna Jagpal, ', 'Iñaki Godoy']","3,123"
9952,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,"After an experimental gene therapy turns them into monsters, three twenty-somethings band together to hunt down the scientist responsible and force him to make them human again.","['Morgan Taylor Campbell, ', 'Italia Ricci, ', 'Rhianna Jagpal, ', 'Iñaki Godoy']","3,130"


In [297]:
df[df['title'] == 'Beauty and the Beast']

,title,year,certificate,duration,genre,rating,description,stars,votes
182,Beauty and the Beast,(I) (2017),PG,129 min,"Adventure, Family, Fantasy",7.1,"A selfish Prince is cursed to become a monster for the rest of his life, unless he learns to fall in love with a beautiful young woman he keeps prisoner.","['Bill Condon', '| ', ' Stars:', 'Emma Watson, ', 'Dan Stevens, ', 'Luke Evans, ', 'Josh Gad']","306,985"
416,Beauty and the Beast,(1991),G,84 min,"Animation, Family, Fantasy",8.0,A prince cursed to spend his days as a hideous monster sets out to regain his humanity by earning a young woman's love.,"['Gary Trousdale, ', 'Kirk Wise', '| ', ' Stars:', ""Paige O'Hara, "", 'Robby Benson, ', 'Jesse Corti, ', 'Rex Everhart']","452,086"


In [298]:
df[df['title'] == 'The Walking Dead']

,title,year,certificate,duration,genre,rating,description,stars,votes
12,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melissa McBride, ', 'Lauren Cohan']","970,060"
9953,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melissa McBride, ', 'Lauren Cohan']","970,067"


### *-->Can not drop duplicates with only column as reference because few movies has same title but rest of the columns are different*

## Deleting duplicates

In [299]:
df.duplicated(subset =['title','year', 'certificate', 'duration', 'genre','stars']).sum()

548

In [300]:
df

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,"Decades after their 1984 All Valley Karate Tournament bout, a middle-aged Daniel LaRusso and Johnny Lawrence find themselves martial-arts rivals again.","['Ralph Macchio, ', 'William Zabka, ', 'Courtney Henggeler, ', 'Xolo Maridueña']","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of Queen Elizabeth II's reign and the events that shaped the second half of the twentieth century.,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda Staunton, ', 'Matt Smith']","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer Jimmy McGill before his fateful run-in with Walter White and Jesse Pinkman.,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan Banks, ', 'Patrick Fabian']","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,"When a psychiatrist shelters a mysterious cult escapee, her world is turned upside down as the girl's arrival threatens to tear her own family apart.","['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo Celasco, ', 'Madeleine Arthur']","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,"A Street Kid trying to survive in a technology and body modification-obsessed city of the future. Having everything to lose, he chooses to stay alive by becoming an Edgerunner, a Mercenary outlaw also known as a Cyberpunk.","['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi Lo, ', 'Aoi Yûki']","15,413"
...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,"After an experimental gene therapy turns them into monsters, three twenty-somethings band together to hunt down the scientist responsible and force him to make them human again.","['Morgan Taylor Campbell, ', 'Italia Ricci, ', 'Rhianna Jagpal, ', 'Iñaki Godoy']","3,130"
9953,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melissa McBride, ', 'Lauren Cohan']","970,067"
9954,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of Queen Elizabeth II's reign and the events that shaped the second half of the twentieth century.,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda Staunton, ', 'Matt Smith']","199,898"
9955,Supernatural,(2005–2020),TV-14,44 min,"Drama, Fantasy, Horror",8.4,"Two brothers follow their father's footsteps as hunters, fighting evil supernatural beings of many kinds, including monsters, demons and gods that roam the earth.","['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim Beaver, ', 'Misha Collins']","439,601"


In [301]:
rows_with_duplicates = df[df.duplicated(subset = ['title','year', 'certificate', 'duration', 'genre','stars'])]
rows_with_duplicates

,title,year,certificate,duration,genre,rating,description,stars,votes
7768,Big Vape: The Incendiary Rise of Juul,NaN,NaN,NaN,Documentary,NaN,Add a Plot,[],NaN
8037,Avatar: The Last Airbender,(2005–2008),TV-Y7,25 min,"Animation, Action, Adventure",7.7,"Aang, Katara, and Sokka have their friendship tested by a charismatic Rebel Leader who is fighting against the Fire Nation.","['Dave Filoni', '| ', ' Stars:', 'Zach Tyler Eisen, ', 'Mae Whitman, ', 'Jack De Sena, ', 'Dee Bradley Baker']","3,993"
8041,Avatar: The Last Airbender,(2005–2008),TV-Y7,24 min,"Animation, Action, Adventure",7.9,Aang is upset when he finds an old Air Temple has been desecrated by an old inventor and his son.,"['Dave Filoni', '| ', ' Stars:', 'Zach Tyler Eisen, ', 'Mae Whitman, ', 'Jack De Sena, ', 'Dee Bradley Baker']","3,682"
8052,The Boondocks,(2005–2014),TV-MA,22 min,"Animation, Action, Comedy",7.8,Riley convinces Granddad to pretend he is blind so that Xzibit will pimp their ride. Huey is being stalked by a government agent. Or is it just his imagination?,"['Anthony Bell, ', 'Lesean Thomas', '| ', ' Stars:', 'Regina King, ', 'John Witherspoon, ', 'Cedric Yarbrough, ', 'Gary Anthony Williams']",361
8054,The Boondocks,(2005–2014),TV-MA,22 min,"Animation, Action, Comedy",8.3,Huey disagrees with Riley - and apparently most Blacks - regarding the R. Kelly case.,"['Anthony Bell', '| ', ' Stars:', 'Regina King, ', 'John Witherspoon, ', 'Cedric Yarbrough, ', 'Gary Anthony Williams']",546
...,...,...,...,...,...,...,...,...,...
9952,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,"After an experimental gene therapy turns them into monsters, three twenty-somethings band together to hunt down the scientist responsible and force him to make them human again.","['Morgan Taylor Campbell, ', 'Italia Ricci, ', 'Rhianna Jagpal, ', 'Iñaki Godoy']","3,130"
9953,The Walking Dead,(2010–2022),TV-MA,44 min,"Drama, Horror, Thriller",8.1,Sheriff Deputy Rick Grimes wakes up from a coma to learn the world is in ruins and must lead a group of survivors to stay alive.,"['Andrew Lincoln, ', 'Norman Reedus, ', 'Melissa McBride, ', 'Lauren Cohan']","970,067"
9954,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of Queen Elizabeth II's reign and the events that shaped the second half of the twentieth century.,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda Staunton, ', 'Matt Smith']","199,898"
9955,Supernatural,(2005–2020),TV-14,44 min,"Drama, Fantasy, Horror",8.4,"Two brothers follow their father's footsteps as hunters, fighting evil supernatural beings of many kinds, including monsters, demons and gods that roam the earth.","['Jared Padalecki, ', 'Jensen Ackles, ', 'Jim Beaver, ', 'Misha Collins']","439,601"


In [302]:
df = df.drop_duplicates(subset=['title','year', 'certificate', 'duration', 'genre','stars'])

In [303]:
df

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,"Decades after their 1984 All Valley Karate Tournament bout, a middle-aged Daniel LaRusso and Johnny Lawrence find themselves martial-arts rivals again.","['Ralph Macchio, ', 'William Zabka, ', 'Courtney Henggeler, ', 'Xolo Maridueña']","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of Queen Elizabeth II's reign and the events that shaped the second half of the twentieth century.,"['Claire Foy, ', 'Olivia Colman, ', 'Imelda Staunton, ', 'Matt Smith']","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer Jimmy McGill before his fateful run-in with Walter White and Jesse Pinkman.,"['Bob Odenkirk, ', 'Rhea Seehorn, ', 'Jonathan Banks, ', 'Patrick Fabian']","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,"When a psychiatrist shelters a mysterious cult escapee, her world is turned upside down as the girl's arrival threatens to tear her own family apart.","['Emily Deschanel, ', 'Sam Jaeger, ', 'Gerardo Celasco, ', 'Madeleine Arthur']","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,"A Street Kid trying to survive in a technology and body modification-obsessed city of the future. Having everything to lose, he chooses to stay alive by becoming an Edgerunner, a Mercenary outlaw also known as a Cyberpunk.","['Zach Aguilar, ', 'Kenichiro Ohashi, ', 'Emi Lo, ', 'Aoi Yûki']","15,413"
...,...,...,...,...,...,...,...,...,...
9914,The Circle,(2020– ),TV-MA,63 min,"Game-Show, Reality-TV",7.4,The experiment starts as the players craft their profiles and begin connecting -- for better or worse.,"['Michelle Buteau, ', 'Karyn Blanco, ', 'Sammie Cimarelli, ', 'Antonio DePina']",216
9915,A Suitable Boy,(2020),TV-14,58 min,"Drama, Romance",6.3,"Lata continues to pursue a relationship with Kabir Durrani, despite knowing her mother would disapprove of the match. When news that her daughter has been seen with an unsuitable boy reaches Rupa she takes action to keep them apart.","['Mira Nair', '| ', ' Stars:', 'Tanya Maniktala, ', 'Danesh Razvi, ', 'Boloram Das, ', 'Mahira Kakkar']",172
9917,Black Summer,(2019–2021),TV-MA,44 min,"Action, Drama, Horror",7.5,"Life and death spin savagely out of control. Familiar faces scramble to survive the brutality, and wrenching betrayal lurks around every corner.","['John Hyams', '| ', ' Stars:', 'Jaime King, ', 'Justin Chu Cary, ', 'Christine Lee, ', 'Zoe Marlett']","1,075"
9919,Star Trek: Discovery,(2017– ),TV-14,47 min,"Action, Adventure, Drama",5.7,"The U.S.S. Discovery crew travel to Book's home planet to help rescue it from Osyraa, the formidable leader of the Emerald Chain; Stamets and Adira continue their search for valuable information on the origin of the Burn.","['Jonathan Frakes', '| ', ' Stars:', 'Sonequa Martin-Green, ', 'Doug Jones, ', 'Anthony Rapp, ', 'Mary Wiseman']","3,268"


## Rechecking after deleation

In [304]:
df[df['title'] == 'The Imperfects']

,title,year,certificate,duration,genre,rating,description,stars,votes
8,The Imperfects,(2022– ),TV-MA,45 min,"Action, Adventure, Drama",6.3,"After an experimental gene therapy turns them into monsters, three twenty-somethings band together to hunt down the scientist responsible and force him to make them human again.","['Morgan Taylor Campbell, ', 'Italia Ricci, ', 'Rhianna Jagpal, ', 'Iñaki Godoy']","3,123"


In [305]:
df[df['title'] == 'Beauty and the Beast']

,title,year,certificate,duration,genre,rating,description,stars,votes
182,Beauty and the Beast,(I) (2017),PG,129 min,"Adventure, Family, Fantasy",7.1,"A selfish Prince is cursed to become a monster for the rest of his life, unless he learns to fall in love with a beautiful young woman he keeps prisoner.","['Bill Condon', '| ', ' Stars:', 'Emma Watson, ', 'Dan Stevens, ', 'Luke Evans, ', 'Josh Gad']","306,985"
416,Beauty and the Beast,(1991),G,84 min,"Animation, Family, Fantasy",8.0,A prince cursed to spend his days as a hideous monster sets out to regain his humanity by earning a young woman's love.,"['Gary Trousdale, ', 'Kirk Wise', '| ', ' Stars:', ""Paige O'Hara, "", 'Robby Benson, ', 'Jesse Corti, ', 'Rex Everhart']","452,086"


In [306]:
df.duplicated(subset =['title','year', 'certificate', 'duration', 'genre','stars']).sum()

0

## Data Cleaning

In [307]:
def clean_text(stars):
    result = str(stars).lower()
    return(result.replace(' ',''))

In [308]:
df['stars'] = df['stars'].apply(clean_text)

C:\Users\Admin\AppData\Local\Temp\ipykernel_31484\1540262484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stars'] = df['stars'].apply(clean_text)


In [309]:
df

,title,year,certificate,duration,genre,rating,description,stars,votes
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,"Decades after their 1984 All Valley Karate Tournament bout, a middle-aged Daniel LaRusso and Johnny Lawrence find themselves martial-arts rivals again.","['ralphmacchio,','williamzabka,','courtneyhenggeler,','xolomaridueña']","177,031"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of Queen Elizabeth II's reign and the events that shaped the second half of the twentieth century.,"['clairefoy,','oliviacolman,','imeldastaunton,','mattsmith']","199,885"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer Jimmy McGill before his fateful run-in with Walter White and Jesse Pinkman.,"['bobodenkirk,','rheaseehorn,','jonathanbanks,','patrickfabian']","501,384"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,"When a psychiatrist shelters a mysterious cult escapee, her world is turned upside down as the girl's arrival threatens to tear her own family apart.","['emilydeschanel,','samjaeger,','gerardocelasco,','madeleinearthur']","9,773"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,"A Street Kid trying to survive in a technology and body modification-obsessed city of the future. Having everything to lose, he chooses to stay alive by becoming an Edgerunner, a Mercenary outlaw also known as a Cyberpunk.","['zachaguilar,','kenichiroohashi,','emilo,','aoiyûki']","15,413"
...,...,...,...,...,...,...,...,...,...
9914,The Circle,(2020– ),TV-MA,63 min,"Game-Show, Reality-TV",7.4,The experiment starts as the players craft their profiles and begin connecting -- for better or worse.,"['michellebuteau,','karynblanco,','sammiecimarelli,','antoniodepina']",216
9915,A Suitable Boy,(2020),TV-14,58 min,"Drama, Romance",6.3,"Lata continues to pursue a relationship with Kabir Durrani, despite knowing her mother would disapprove of the match. When news that her daughter has been seen with an unsuitable boy reaches Rupa she takes action to keep them apart.","['miranair','|','stars:','tanyamaniktala,','daneshrazvi,','boloramdas,','mahirakakkar']",172
9917,Black Summer,(2019–2021),TV-MA,44 min,"Action, Drama, Horror",7.5,"Life and death spin savagely out of control. Familiar faces scramble to survive the brutality, and wrenching betrayal lurks around every corner.","['johnhyams','|','stars:','jaimeking,','justinchucary,','christinelee,','zoemarlett']","1,075"
9919,Star Trek: Discovery,(2017– ),TV-14,47 min,"Action, Adventure, Drama",5.7,"The U.S.S. Discovery crew travel to Book's home planet to help rescue it from Osyraa, the formidable leader of the Emerald Chain; Stamets and Adira continue their search for valuable information on the origin of the Burn.","['jonathanfrakes','|','stars:','sonequamartin-green,','dougjones,','anthonyrapp,','marywiseman']","3,268"


In [310]:
df['Key_words'] = ""
df.head()
for index, row in df.iterrows():
    plot = row['description']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()
    
    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)
    
    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    
    # Convert the dictionary keys to a list
    key_words_list = list(key_words_dict_scores.keys())
    
    
    # assigning the key words to the new column for the corresponding movie
    df.at[index, 'Key_words'] = key_words_list

C:\Users\Admin\AppData\Local\Temp\ipykernel_31484\1942813525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Key_words'] = ""


In [311]:
df

,title,year,certificate,duration,genre,rating,description,stars,votes,Key_words
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,"Decades after their 1984 All Valley Karate Tournament bout, a middle-aged Daniel LaRusso and Johnny Lawrence find themselves martial-arts rivals again.","['ralphmacchio,','williamzabka,','courtneyhenggeler,','xolomaridueña']","177,031","[decades, 1984, valley, karate, tournament, bout, middle, aged, daniel, larusso, johnny, lawrence, find, martial, arts, rivals]"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of Queen Elizabeth II's reign and the events that shaped the second half of the twentieth century.,"['clairefoy,','oliviacolman,','imeldastaunton,','mattsmith']","199,885","[follows, political, rivalries, romance, queen, elizabeth, ii, reign, events, shaped, second, half, twentieth, century]"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer Jimmy McGill before his fateful run-in with Walter White and Jesse Pinkman.,"['bobodenkirk,','rheaseehorn,','jonathanbanks,','patrickfabian']","501,384","[trials, tribulations, criminal, lawyer, jimmy, mcgill, fateful, run, walter, white, jesse, pinkman]"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,"When a psychiatrist shelters a mysterious cult escapee, her world is turned upside down as the girl's arrival threatens to tear her own family apart.","['emilydeschanel,','samjaeger,','gerardocelasco,','madeleinearthur']","9,773","[psychiatrist, shelters, mysterious, cult, escapee, world, turned, upside, girl, arrival, threatens, tear, family, apart]"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,"A Street Kid trying to survive in a technology and body modification-obsessed city of the future. Having everything to lose, he chooses to stay alive by becoming an Edgerunner, a Mercenary outlaw also known as a Cyberpunk.","['zachaguilar,','kenichiroohashi,','emilo,','aoiyûki']","15,413","[street, kid, trying, survive, technology, body, modification, obsessed, city, future, everything, lose, chooses, stay, alive, becoming, edgerunner, mercenary, outlaw, also, known, cyberpunk]"
...,...,...,...,...,...,...,...,...,...,...
9914,The Circle,(2020– ),TV-MA,63 min,"Game-Show, Reality-TV",7.4,The experiment starts as the players craft their profiles and begin connecting -- for better or worse.,"['michellebuteau,','karynblanco,','sammiecimarelli,','antoniodepina']",216,"[experiment, starts, players, craft, profiles, begin, connecting, --, better, worse]"
9915,A Suitable Boy,(2020),TV-14,58 min,"Drama, Romance",6.3,"Lata continues to pursue a relationship with Kabir Durrani, despite knowing her mother would disapprove of the match. When news that her daughter has been seen with an unsuitable boy reaches Rupa she takes action to keep them apart.","['miranair','|','stars:','tanyamaniktala,','daneshrazvi,','boloramdas,','mahirakakkar']",172,"[lata, continues, pursue, relationship, kabir, durrani, despite, knowing, mother, would, disapprove, match, news, daughter, seen, unsuitable, boy, reaches, rupa, takes, action, keep, apart]"
9917,Black Summer,(2019–2021),TV-MA,44 min,"Action, Drama, Horror",7.5,"Life and death spin savagely out of control. Familiar faces scramble to survive the brutality, and wrenching betrayal lurks around every corner.","['johnhyams','|','stars:','jaimeking,','justinchucary,','christinelee,','zoemarlett']","1,075","[life, death, spin, savagely, control, familiar, faces, scramble, survive, brutality, wrenching, betrayal, lurks, around, every, corner]"
9919,Star Trek: Discovery,(2017– ),TV-14,47 min,"Action, Adventure, Drama",5.7,"The U.S.S. Discovery crew travel to Book's home planet to help rescue it from Osyraa, the formidable leader of the Emerald Chain; Stamets and Adira continue their search for valuable information on the origin of the Burn.","['jonathanfrakes',

In [312]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9409 entries, 0 to 9920
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        9409 non-null   object 
 1   year         8883 non-null   object 
 2   certificate  6078 non-null   object 
 3   duration     7650 non-null   object 
 4   genre        9336 non-null   object 
 5   rating       8311 non-null   float64
 6   description  9409 non-null   object 
 7   stars        9409 non-null   object 
 8   votes        8311 non-null   object 
 9   Key_words    9409 non-null   object 
dtypes: float64(1), object(9)
memory usage: 1.0+ MB


In [313]:
df['genre_description'] = ""
df.head()
for index, row in df.iterrows():
    plot = row[['description','genre']]
    
     # Concatenate 'description' and 'genre' columns into a single string
    plot = row['description'] + str(row['genre'])
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()
    
    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)
    
    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    
    # Convert the dictionary keys to a list
    key_words_list = list(key_words_dict_scores.keys())
    
    
    # assigning the key words to the new column for the corresponding movie
    df.at[index, 'genre_description'] = key_words_list

C:\Users\Admin\AppData\Local\Temp\ipykernel_31484\622913089.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genre_description'] = ""


In [314]:
df.drop(columns =['Key_words'], inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_31484\498960258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns =['Key_words'], inplace=True)


In [315]:
df['genre_stars'] = ""
df.head()
for index, row in df.iterrows():
    plot = row[['genre','stars']]
    
     # Concatenate 'description' and 'genre' columns into a single string
    plot = row['stars'] + str(row['genre'])
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()
    
    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)
    
    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    
    # Convert the dictionary keys to a list
    key_words_list = list(key_words_dict_scores.keys())
    
    
    # assigning the key words to the new column for the corresponding movie
    df.at[index, 'genre_stars'] = key_words_list

C:\Users\Admin\AppData\Local\Temp\ipykernel_31484\1124999500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genre_stars'] = ""


In [316]:
df

,title,year,certificate,duration,genre,rating,description,stars,votes,genre_description,genre_stars
0,Cobra Kai,(2018– ),TV-14,30 min,"Action, Comedy, Drama",8.5,"Decades after their 1984 All Valley Karate Tournament bout, a middle-aged Daniel LaRusso and Johnny Lawrence find themselves martial-arts rivals again.","['ralphmacchio,','williamzabka,','courtneyhenggeler,','xolomaridueña']","177,031","[decades, 1984, valley, karate, tournament, bout, middle, aged, daniel, larusso, johnny, lawrence, find, martial, arts, rivals, action, comedy, drama]","[[', ralphmacchio, ,',', williamzabka, courtneyhenggeler, xolomaridueña, '], action, comedy, drama]"
1,The Crown,(2016– ),TV-MA,58 min,"Biography, Drama, History",8.7,Follows the political rivalries and romance of Queen Elizabeth II's reign and the events that shaped the second half of the twentieth century.,"['clairefoy,','oliviacolman,','imeldastaunton,','mattsmith']","199,885","[follows, political, rivalries, romance, queen, elizabeth, ii, reign, events, shaped, second, half, twentieth, century, biography, drama, history]","[[', clairefoy, ,',', oliviacolman, imeldastaunton, mattsmith, '], biography, drama, history]"
2,Better Call Saul,(2015–2022),TV-MA,46 min,"Crime, Drama",8.9,The trials and tribulations of criminal lawyer Jimmy McGill before his fateful run-in with Walter White and Jesse Pinkman.,"['bobodenkirk,','rheaseehorn,','jonathanbanks,','patrickfabian']","501,384","[trials, tribulations, criminal, lawyer, jimmy, mcgill, fateful, run, walter, white, jesse, pinkman, crime, drama]","[[', bobodenkirk, ,',', rheaseehorn, jonathanbanks, patrickfabian, '], crime, drama]"
3,Devil in Ohio,(2022),TV-MA,356 min,"Drama, Horror, Mystery",5.9,"When a psychiatrist shelters a mysterious cult escapee, her world is turned upside down as the girl's arrival threatens to tear her own family apart.","['emilydeschanel,','samjaeger,','gerardocelasco,','madeleinearthur']","9,773","[psychiatrist, shelters, mysterious, cult, escapee, world, turned, upside, girl, arrival, threatens, tear, family, apart, drama, horror, mystery]","[[', emilydeschanel, ,',', samjaeger, gerardocelasco, madeleinearthur, '], drama, horror, mystery]"
4,Cyberpunk: Edgerunners,(2022– ),TV-MA,24 min,"Animation, Action, Adventure",8.6,"A Street Kid trying to survive in a technology and body modification-obsessed city of the future. Having everything to lose, he chooses to stay alive by becoming an Edgerunner, a Mercenary outlaw also known as a Cyberpunk.","['zachaguilar,','kenichiroohashi,','emilo,','aoiyûki']","15,413","[street, kid, trying, survive, technology, body, modification, obsessed, city, future, everything, lose, chooses, stay, alive, becoming, edgerunner, mercenary, outlaw, also, known, cyberpunk, animation, action, adventure]","[[', zachaguilar, ,',', kenichiroohashi, emilo, aoiyûki, '], animation, action, adventure]"
...,...,...,...,...,...,...,...,...,...,...,...
9914,The Circle,(2020– ),TV-MA,63 min,"Game-Show, Reality-TV",7.4,The experiment starts as the players craft their profiles and begin connecting -- for better or worse.,"['michellebuteau,','karynblanco,','sammiecimarelli,','antoniodepina']",216,"[experiment, starts, players, craft, profiles, begin, connecting, --, better, worse, game, show, reality, tv]","[[', michellebuteau, ,',', karynblanco, sammiecimarelli, antoniodepina, '], game, show, reality, tv]"
9915,A Suitable Boy,(2020),TV-14,58 min,"Drama, Romance",6.3,"Lata continues to pursue a relationship with Kabir Durrani, despite knowing her mother would disapprove of the match. When news that her daughter has been seen with an unsuitable boy reaches Rupa she takes action to keep them apart.","['miranair','|','stars:','tanyamaniktala,','daneshrazvi,','boloramdas,','mahirakakkar']",172,"[lata, continues, pursue, relationship, kabir, durrani, despite, knowing, mother, would, disapprove, match, news, daughter, seen, unsuitable, boy, reaches, rupa, takes, action, keep, apart, drama, romance]","[[', mira

In [318]:
pd.set_option('display.max_colwidth', None)
df['genre_stars']

0                                             [[', ralphmacchio, ,',', williamzabka, courtneyhenggeler, xolomaridueña, '], action, comedy, drama]
1                                                   [[', clairefoy, ,',', oliviacolman, imeldastaunton, mattsmith, '], biography, drama, history]
2                                                            [[', bobodenkirk, ,',', rheaseehorn, jonathanbanks, patrickfabian, '], crime, drama]
3                                              [[', emilydeschanel, ,',', samjaeger, gerardocelasco, madeleinearthur, '], drama, horror, mystery]
4                                                      [[', zachaguilar, ,',', kenichiroohashi, emilo, aoiyûki, '], animation, action, adventure]
                                                                          ...                                                                    
9914                                         [[', michellebuteau, ,',', karynblanco, sammiecimarelli, antoniodepina, '], gam

## Vectorizing the Dataframe

In [319]:
df2 = df.drop(['certificate','year','genre','description'],axis=1)

In [320]:
df2

,title,duration,rating,stars,votes,genre_description,genre_stars
0,Cobra Kai,30 min,8.5,"['ralphmacchio,','williamzabka,','courtneyhenggeler,','xolomaridueña']","177,031","[decades, 1984, valley, karate, tournament, bout, middle, aged, daniel, larusso, johnny, lawrence, find, martial, arts, rivals, action, comedy, drama]","[[', ralphmacchio, ,',', williamzabka, courtneyhenggeler, xolomaridueña, '], action, comedy, drama]"
1,The Crown,58 min,8.7,"['clairefoy,','oliviacolman,','imeldastaunton,','mattsmith']","199,885","[follows, political, rivalries, romance, queen, elizabeth, ii, reign, events, shaped, second, half, twentieth, century, biography, drama, history]","[[', clairefoy, ,',', oliviacolman, imeldastaunton, mattsmith, '], biography, drama, history]"
2,Better Call Saul,46 min,8.9,"['bobodenkirk,','rheaseehorn,','jonathanbanks,','patrickfabian']","501,384","[trials, tribulations, criminal, lawyer, jimmy, mcgill, fateful, run, walter, white, jesse, pinkman, crime, drama]","[[', bobodenkirk, ,',', rheaseehorn, jonathanbanks, patrickfabian, '], crime, drama]"
3,Devil in Ohio,356 min,5.9,"['emilydeschanel,','samjaeger,','gerardocelasco,','madeleinearthur']","9,773","[psychiatrist, shelters, mysterious, cult, escapee, world, turned, upside, girl, arrival, threatens, tear, family, apart, drama, horror, mystery]","[[', emilydeschanel, ,',', samjaeger, gerardocelasco, madeleinearthur, '], drama, horror, mystery]"
4,Cyberpunk: Edgerunners,24 min,8.6,"['zachaguilar,','kenichiroohashi,','emilo,','aoiyûki']","15,413","[street, kid, trying, survive, technology, body, modification, obsessed, city, future, everything, lose, chooses, stay, alive, becoming, edgerunner, mercenary, outlaw, also, known, cyberpunk, animation, action, adventure]","[[', zachaguilar, ,',', kenichiroohashi, emilo, aoiyûki, '], animation, action, adventure]"
...,...,...,...,...,...,...,...
9914,The Circle,63 min,7.4,"['michellebuteau,','karynblanco,','sammiecimarelli,','antoniodepina']",216,"[experiment, starts, players, craft, profiles, begin, connecting, --, better, worse, game, show, reality, tv]","[[', michellebuteau, ,',', karynblanco, sammiecimarelli, antoniodepina, '], game, show, reality, tv]"
9915,A Suitable Boy,58 min,6.3,"['miranair','|','stars:','tanyamaniktala,','daneshrazvi,','boloramdas,','mahirakakkar']",172,"[lata, continues, pursue, relationship, kabir, durrani, despite, knowing, mother, would, disapprove, match, news, daughter, seen, unsuitable, boy, reaches, rupa, takes, action, keep, apart, drama, romance]","[[', miranair, ','|',', stars, :',', tanyamaniktala, ,',', daneshrazvi, boloramdas, mahirakakkar, '], drama, romance]"
9917,Black Summer,44 min,7.5,"['johnhyams','|','stars:','jaimeking,','justinchucary,','christinelee,','zoemarlett']","1,075","[life, death, spin, savagely, control, familiar, faces, scramble, survive, brutality, wrenching, betrayal, lurks, around, every, corner, action, drama, horror]","[[', johnhyams, ','|',', stars, :',', jaimeking, ,',', justinchucary, christinelee, zoemarlett, '], action, drama, horror]"
9919,Star Trek: Discovery,47 min,5.7,"['jonathanfrakes','|','stars:','sonequamartin-green,','dougjones,','anthonyrapp,','marywiseman']","3,268","[u, discovery, crew, travel, book, home, planet, help, rescue, osyraa, formidable, leader, emerald, chain, stamets, adira, continue, search, valuable, information, origin, burn, action, adventure, drama]","[[', jonathanfrakes, ','|',', stars, :',', sonequamartin, green, ,',', dougjones, anthonyrapp, marywiseman, '], action, adventure, drama]"


In [321]:
df2['genre_description']

0                                                                              [decades, 1984, valley, karate, tournament, bout, middle, aged, daniel, larusso, johnny, lawrence, find, martial, arts, rivals, action, comedy, drama]
1                                                                                  [follows, political, rivalries, romance, queen, elizabeth, ii, reign, events, shaped, second, half, twentieth, century, biography, drama, history]
2                                                                                                                  [trials, tribulations, criminal, lawyer, jimmy, mcgill, fateful, run, walter, white, jesse, pinkman, crime, drama]
3                                                                                   [psychiatrist, shelters, mysterious, cult, escapee, world, turned, upside, girl, arrival, threatens, tear, family, apart, drama, horror, mystery]
4       [street, kid, trying, survive, technology, body, modification, obsessed,

In [322]:
from sklearn.feature_extraction.text import CountVectorizer

# Assuming df2['genre_description'] contains lists
# Convert lists to strings
df2['genre_description'] = df2['genre_description'].apply(lambda x: ' '.join(x))

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the text data
vectorized = vectorizer.fit_transform(df2['genre_description'])

In [323]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)

In [324]:
print(similarities)

[[1.         0.05564149 0.06131393 ... 0.10526316 0.09365858 0.05923489]
 [0.05564149 1.         0.06482037 ... 0.05564149 0.04950738 0.        ]
 [0.06131393 0.06482037 1.         ... 0.06131393 0.05455447 0.        ]
 ...
 [0.10526316 0.05564149 0.06131393 ... 1.         0.09365858 0.        ]
 [0.09365858 0.04950738 0.05455447 ... 0.09365858 1.         0.        ]
 [0.05923489 0.         0.         ... 0.         0.         1.        ]]


In [326]:
df = pd.DataFrame(similarities, columns=df['title'], index=df['title']).reset_index()

df.head(20)

title,title,Cobra Kai,The Crown,Better Call Saul,Devil in Ohio,Cyberpunk: Edgerunners,The Sandman,Rick and Morty,Breaking Bad,The Imperfects,...,Age of Samurai: Battle for Japan,Vikings,Money Heist,Winning Time: The Rise of the Lakers Dynasty,Unstoppable,The Circle,A Suitable Boy,Black Summer,Star Trek: Discovery,The Upshaws
0,Cobra Kai,1.000000,0.055641,0.061314,0.055641,0.045883,0.114708,0.069171,0.052632,0.105263,...,0.051299,0.093659,0.108148,0.057354,0.118470,0.0,0.091766,0.105263,0.093659,0.059235
1,The Crown,0.055641,1.000000,0.064820,0.058824,0.000000,0.060634,0.073127,0.055641,0.055641,...,0.000000,0.049507,0.057166,0.121268,0.062622,0.0,0.097014,0.055641,0.049507,0.000000
2,Better Call Saul,0.061314,0.064820,1.000000,0.064820,0.000000,0.066815,0.000000,0.122628,0.061314,...,0.000000,0.054554,0.125988,0.066815,0.069007,0.0,0.053452,0.061314,0.054554,0.000000
3,Devil in Ohio,0.055641,0.058824,0.064820,1.000000,0.000000,0.121268,0.000000,0.111283,0.055641,...,0.000000,0.049507,0.114332,0.060634,0.062622,0.0,0.097014,0.111283,0.049507,0.062622
4,Cyberpunk: Edgerunners,0.045883,0.000000,0.000000,0.000000,1.000000,0.000000,0.120605,0.045883,0.091766,...,0.089443,0.081650,0.047140,0.000000,0.000000,0.0,0.040000,0.091766,0.081650,0.000000
5,The Sandman,0.114708,0.060634,0.066815,0.121268,0.000000,1.000000,0.000000,0.057354,0.057354,...,0.000000,0.102062,0.058926,0.062500,0.064550,0.0,0.050000,0.114708,0.051031,0.000000
6,Rick and Morty,0.069171,0.073127,0.000000,0.000000,0.120605,0.000000,1.000000,0.000000,0.138343,...,0.067420,0.061546,0.000000,0.000000,0.077850,0.0,0.000000,0.000000,0.061546,0.077850
7,Breaking Bad,0.052632,0.055641,0.122628,0.111283,0.045883,0.057354,0.000000,1.000000,0.105263,...,0.000000,0.046829,0.108148,0.057354,0.059235,0.0,0.045883,0.052632,0.046829,0.059235
8,The Imperfects,0.105263,0.055641,0.061314,0.055641,0.091766,0.057354,0.138343,0.105263,1.000000,...,0.102598,0.140488,0.108148,0.057354,0.059235,0.0,0.091766,0.105263,0.140488,0.000000
9,Blonde,0.076472,0.161690,0.089087,0.161690,0.000000,0.083333,0.000000,0.076472,0.076472,...,0.000000,0.068041,0.078567,0.166667,0.086066,0.0,0.066667,0.152944,0.068041,0.000000


In [328]:
input_title = 'The Imperfects'
recommendations = pd.DataFrame(df.nlargest(11,input_book)['title'])
recommendations = recommendations[recommendations['title']!=input_title]
recommendations

,title
2270,Assassin's Creed
4798,Power Rangers
566,Sweet Tooth
8466,Arthdal Chronicles
923,Colony
2185,The Last Bus
2663,Berserk: The Golden Age Arc II - The Battle for Doldrey
7338,Supercrooks
279,Supergirl
3504,Oni: Thunder God's Tale


In [330]:
input_title = 'Breaking Bad'
recommendations = pd.DataFrame(df.nlargest(11,input_book)['title'])
recommendations = recommendations[recommendations['title']!=input_title]
recommendations

,title
3224,Kakegurui
1625,Black Space
3202,Dirty Teacher
5106,Quiet Victory: The Charlie Wedemeyer Story
1099,You Get Me
5379,The Unguarded Moment
4416,Death Clique
437,Hell or High Water
5057,Hear No Evil
6388,Rattle the Cage
